In [ ]:
from PIL import Image, ImageDraw
import matplotlib.patches as patches
import math
import os
import random
from pathlib import Path
import numpy as np
import yaml
import glob
import cv2
import torch
from pathlib import Path
import pandas as pd
import os
import shutil

def myFig(img, bbox_list, drawB = True) :
    image = Image.fromarray(img,"RGB")
    draw = ImageDraw.Draw(image)

    if drawB :
        xmin = bbox[0] 
        ymin = bbox[1]
        xmax = bbox[2]
        ymax = bbox[3]

        draw.rectangle((xmin, ymin, xmax, ymax), outline=(255,0,0), width = 1) # bounding box
#     display(image)
    return image

def make_json():
    coco_json_train = {}
    coco_json_train['licenses'] = [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/','id': 1,'name': 'Attribution-NonCommercial-ShareAlike License'}]
    coco_json_train['info'] = {'description': 'COCO apple Dataset','url': '','version': '','year': 2023,'contributor': 'COCO Consortium','date_created': '2023/04/11'}
    coco_json_train['annotations'] = []
    coco_json_train['categories'] = [{'supercategory': 'apple', 'id': 0, 'name': 'apple'}]
    coco_json_train['images'] = []


    coco_json_val = {}
    coco_json_val['licenses'] = [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/','id': 1,'name': 'Attribution-NonCommercial-ShareAlike License'}]
    coco_json_val['info'] = {'description': 'COCO apple Dataset','url': '','version': '','year': 2023,'contributor': 'COCO Consortium','date_created': '2023/04/11'}
    coco_json_val['annotations'] = []
    coco_json_val['categories'] = [{'supercategory': 'apple', 'id': 0, 'name': 'apple'}]
    coco_json_val['images'] = []
    
    return coco_json_train, coco_json_val


def make_dir(path):
    if not os.path.exists(path):
        print(path)
        make_dir(os.path.dirname(path))
        os.makedirs(path, exist_ok=True)

        
def myLoadImage(im_path,img_size, resize=True, augment=True) :
    im = cv2.imread(im_path)
    print(im.shape)
    h0, w0 = im.shape[:2]
    if resize : 
        r = img_size / max(h0, w0)
        if r != 1:  # if sizes are not equal
            interp = cv2.INTER_LINEAR if augment else cv2.INTER_AREA  # random.choice(self.rand_interp_methods)
            im = cv2.resize(im, (int(w0 * r), int(h0 * r)), interpolation=interp)
    return im, (h0, w0), im.shape[:2] 

def xywhn2xyxy(x, w=640, h=640, padw=0, padh=0):
    # Convert nx4 boxes from [x, y, w, h] normalized to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = w * (x[:, 0] - x[:, 2] / 2) + padw  # top left x
    y[:, 1] = h * (x[:, 1] - x[:, 3] / 2) + padh  # top left y
    y[:, 2] = w * (x[:, 0] + x[:, 2] / 2) + padw  # bottom right x
    y[:, 3] = h * (x[:, 1] + x[:, 3] / 2) + padh  # bottom right y
    return y

def convert_coordinates(x1, y1, x2, y2, img_width, img_height, cate):
    # 바운딩 박스의 중심 좌표와 너비, 높이 계산
    box_width = x2 - x1
    box_height = y2 - y1
    x_center = (x1 + x2) / 2.0
    y_center = (y1 + y2) / 2.0
    
    # 이미지의 너비와 높이로 나누어서 정규화된 값 계산
    x_center /= img_width
    y_center /= img_height
    box_width /= img_width
    box_height /= img_height
    
    # 결과를 문자열로 반환
    return f"{cate} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"

In [ ]:
from imantics import Polygons, Mask
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
import pickle
import os
import random

def test(back_img_list, save_img_path, save_label_path, save_annotations_path,coco_json,id_cnt,t):
    category_id = 0
    fileEx = r'.pickle'
    pickle_root_path = './pickle'
    pickle_list = [file for file in os.listdir(pickle_root_path) if file.endswith(fileEx)]
    
    for idx, back_img_file in enumerate(back_img_list) : 
        print(back_img_file)
        back_img = cv2.imread(back_img_file)
        s_lm = int(min(back_img.shape[0],back_img.shape[1])-100)
        resize_size = random.randrange(30,255)
        
        pickle_file = pickle_list[random.randrange(0,len(pickle_list)-1)]
        pickle_path = f'{pickle_root_path}/{pickle_file}'
        with open(f'{pickle_path}', 'rb') as f:
            data = pickle.load(f)

        mask = data['mask']
        mask = cv2.resize(mask, (resize_size, resize_size), interpolation=cv2.INTER_LANCZOS4)

        target_img_name = pickle_file.split('.')[0]

        target_img_path = f'/data/ij/edge/LDC/good_apple/{target_img_name}.jpg'
        target_img = cv2.imread(target_img_path)
        target_img = cv2.resize(target_img, (resize_size, resize_size), interpolation=cv2.INTER_LANCZOS4)
        
        
        t_s = target_img.shape[0]

        r_degree = random.randrange(0,360)
        b = cv2.getRotationMatrix2D((t_s/2,t_s/2), r_degree,1)
        mask = cv2.warpAffine(mask, b,(t_s,t_s),borderValue = 255)
        target_img = cv2.warpAffine(target_img, b,(t_s,t_s),borderValue = 255)
        mask[mask >= 50] = 255
        mask[mask < 50] = 0


        mask_inv = cv2.bitwise_not(mask)

        height1, width1 = target_img.shape[:2]
        height2, width2 = back_img.shape[:2]

        x_limit = int(back_img.shape[1])-resize_size
        y_limit = int(back_img.shape[0])-resize_size
        x1 = random.randrange(0,x_limit)
        y1 = random.randrange(0,y_limit)
        x2 = x1 + width1
        y2 = y1 + height1

        roi = back_img[y1:y2, x1:x2]
        fg = cv2.bitwise_and(target_img, target_img, mask=mask_inv)
        bg = cv2.bitwise_and(roi, roi, mask=mask)
        back_img[y1:y2, x1:x2] = fg + bg
        bbox = [x1 ,y1, x2, y2]

        
        cv2.imwrite(f'{save_img_path}/{idx+t}.jpg',back_img)

        height, width, _ = back_img.shape
        idx_name = idx + t
        img_res = {'license': 1, 'file_name': f'{idx_name}.jpg','coco_url': '','height': height,'width': width,'date_captured': '','flickr_url': '','id': idx_name}

        coco_json['images'].append(img_res)

        with open(f'{save_label_path}/{idx_name}.txt', "w") as f:

            x1, y1, x2, y2 = bbox[0],bbox[1],bbox[2],bbox[3]

            yolo_bbox = convert_coordinates(x1, y1, x2, y2, width, height, category_id)
            f.write(f"{yolo_bbox}\n")

            coco_bbox = [x1, y1, x2 - x1, y2 - y1]  # COCO bbox 형식으로 변환

            area = coco_bbox[2] * coco_bbox[3]
            res = {'segmentation':[] ,'area': area ,'iscrowd': 0,'image_id': idx_name,'bbox': coco_bbox ,'category_id': category_id,'id': id_cnt}
            coco_json['annotations'].append(res)
            id_cnt += 1



    import json
    with open(save_annotations_path, "w") as f:
        json.dump(coco_json, f)
    
    return id_cnt
        
        



In [ ]:
#폴더만들기

     
root_path = '/data/ij/edge/apple'
if os.path.exists(root_path) : 
    shutil.rmtree(root_path)
else :
    os.mkdir(root_path)

save_img_path = f'{root_path}/images/train2017'
save_img_path_val = f'{root_path}/images/val2017'

save_label_path = f'{root_path}/labels/train2017'
save_label_path_val = f'{root_path}/labels/val2017'
save_annotations_path = f'{root_path}/annotations/'
save_annotations_train = f'{root_path}/annotations/instances_train2017.json'
save_annotations_val = f'{root_path}/annotations/instances_val2017.json'

save_path_list = [save_img_path,save_img_path_val,save_label_path,save_label_path_val,save_annotations_path]

for save_path in save_path_list : 
    make_dir(save_path)


In [ ]:
id_cnt = 1
t = 0
id_cnt= test(train_img_list, save_img_path, save_label_path, save_annotations_train,coco_json_train, id_cnt,t)

#val
t = int(len(train_img_list))
id_cnt= test(val_img_list, save_img_path_val, save_label_path_val, save_annotations_val,coco_json_val, id_cnt,t)
